# AI-Powered Quiz Generator

This notebook demonstrates an AI-powered quiz generator using LangChain and OpenAI's GPT-3.5-turbo model.

## Features
- Generates customized quizzes based on user-specified categories (Geography, Science, Art)
- Uses a knowledge base of facts about various subjects
- Includes evaluation functions to test the quiz generation

## Setup and Configuration

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Load API tokens for our 3rd party APIs

In [3]:
from utils import get_circle_api_key
cci_api_key = get_circle_api_key()

from utils import get_gh_api_key
gh_api_key = get_gh_api_key()

from utils import get_openai_api_key
openai_api_key = get_openai_api_key()

### Set up our github branch

In [4]:
from utils import get_repo_name
course_repo = get_repo_name()
course_repo

'quiz-generator'

In [5]:
from utils import get_branch
course_branch = get_branch()
course_branch

'main'

## The Sample Application: AI-powered Quiz Generator

We are going to build an AI powered quiz generator.

### Create the dataset for the quiz

In [6]:
human_template = "{question}"

quiz_bank = """1. Subject: Leonardo DaVinci
   Categories: Art, Science
   Facts:
    - Painted the Mona Lisa
    - Studied zoology, anatomy, geology, optics
    - Designed a flying machine
  
2. Subject: Paris
   Categories: Art, Geography
   Facts:
    - Location of the Louvre, the museum where the Mona Lisa is displayed
    - Capital of France
    - Most populous city in France
    - Where Radium and Polonium were discovered by scientists Marie and Pierre Curie

3. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berillyum mirror

4. Subject: Starry Night
   Category: Art
   Facts:
    - Painted by Vincent van Gogh in 1889
    - Captures the east-facing view of van Gogh's room in Saint-Rémy-de-Provence

5. Subject: Physics
   Category: Science
   Facts:
    - The sun doesn't change color during sunset.
    - Water slows the speed of light
    - The Eiffel Tower in Paris is taller in the summer than the winter due to expansion of the metal."""

### Build the prompt template

In [7]:
delimiter = "####"

prompt_template = f"""
Follow these steps to generate a customized quiz for the user.
The question will be delimited with four hashtags i.e {delimiter}

The user will provide a category that they want to create a quiz for. Any questions included in the quiz
should only refer to the category.

Step 1:{delimiter} First identify the category user is asking about from the following list:
* Geography
* Science
* Art

Step 2:{delimiter} Determine the subjects to generate questions about. The list of topics are below:

{quiz_bank}

Pick up to two subjects that fit the user's category. 

Step 3:{delimiter} Generate a quiz for the user. Based on the selected subjects generate 3 questions for the user using the facts about the subject.

Use the following format for the quiz:
Question 1:{delimiter} <question 1>

Question 2:{delimiter} <question 2>

Question 3:{delimiter} <question 3>

"""

### Use LangChain to build the prompt template

In [8]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])
# print to observe the content or generated object
chat_prompt

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\nFollow these steps to generate a customized quiz for the user.\nThe question will be delimited with four hashtags i.e ####\n\nThe user will provide a category that they want to create a quiz for. Any questions included in the quiz\nshould only refer to the category.\n\nStep 1:#### First identify the category user is asking about from the following list:\n* Geography\n* Science\n* Art\n\nStep 2:#### Determine the subjects to generate questions about. The list of topics are below:\n\n1. Subject: Leonardo DaVinci\n   Categories: Art, Science\n   Facts:\n    - Painted the Mona Lisa\n    - Studied zoology, anatomy, geology, optics\n    - Designed a flying machine\n\n2. Subject: Paris\n   Categories: Art, Geography\n   Facts:\n    - Location of the Louvre, the museum where the Mona Lisa is

### Choose the LLM

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x112aa83b0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x113008350>, root_client=<openai.OpenAI object at 0x111eaf2f0>, root_async_client=<openai.AsyncOpenAI object at 0x112aa8320>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Set up an output parser

Set up an output parser in LangChain that converts the LLM response into a string.

In [10]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()
output_parser

StrOutputParser()

### Connect the pieces using LCEL (LangChain Expression Language)

In [11]:
chain = chat_prompt | llm | output_parser
chain

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\nFollow these steps to generate a customized quiz for the user.\nThe question will be delimited with four hashtags i.e ####\n\nThe user will provide a category that they want to create a quiz for. Any questions included in the quiz\nshould only refer to the category.\n\nStep 1:#### First identify the category user is asking about from the following list:\n* Geography\n* Science\n* Art\n\nStep 2:#### Determine the subjects to generate questions about. The list of topics are below:\n\n1. Subject: Leonardo DaVinci\n   Categories: Art, Science\n   Facts:\n    - Painted the Mona Lisa\n    - Studied zoology, anatomy, geology, optics\n    - Designed a flying machine\n\n2. Subject: Paris\n   Categories: Art, Geography\n   Facts:\n    - Location of the Louvre, the museum where the Mona Lisa is

### Build the reusable `assistant_chain` function

This function puts together all steps above into a reusable component.

In [12]:
def assistant_chain(
    system_message,
    human_template="{question}",
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    output_parser=StrOutputParser()):
  
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", human_template),
    ])
    return chat_prompt | llm | output_parser

## Evaluations

Now let's create evaluation functions to test our quiz generator.

### Evaluation Function: Expected Words

Create the function `eval_expected_words` to check if the generated quiz contains expected keywords.

In [13]:
def eval_expected_words(
    system_message,
    question,
    expected_words,
    human_template="{question}",
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    output_parser=StrOutputParser()):
    
    assistant = assistant_chain(
        system_message,
        human_template,
        llm,
        output_parser)
    
    answer = assistant.invoke({"question": question})
    
    print(answer)
    
    assert any(word in answer.lower() \
               for word in expected_words), \
        f"Expected the assistant questions to include \
        '{expected_words}', but it did not"

### Test: Generate a quiz about science

In [14]:
question = "Generate a quiz about science."
expected_words = ["davinci", "telescope", "physics", "curie"]

### Run the evaluation

In [15]:
eval_expected_words(
    prompt_template,
    question,
    expected_words
)

Step 1:#### First identify the category user is asking about from the following list:
* Geography
* Science
* Art

User's Category: Science

Step 2:#### Determine the subjects to generate questions about. The list of topics are below:

1. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berillyum mirror

2. Subject: Physics
   Category: Science
   Facts:
    - The sun doesn't change color during sunset.
    - Water slows the speed of light
    - The Eiffel Tower in Paris is taller in the summer than the winter due to expansion of the metal.

Selected Subjects: Telescopes, Physics

Step 3:#### Generate a quiz for the user. Based on the selected subjects generate 3 questions for the user using the facts about the subject.

Question 1:#### What is the James Webb space te

### Evaluation Function: Refusal Test

Create the function `evaluate_refusal` to define a failing test case where the app should decline to answer.

In [16]:
def evaluate_refusal(
    system_message,
    question,
    decline_response,
    human_template="{question}", 
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    output_parser=StrOutputParser()):
    
    assistant = assistant_chain(
        system_message,
        human_template,
        llm,
        output_parser)
  
    answer = assistant.invoke({"question": question})
    print(answer)
  
    assert decline_response.lower() in answer.lower(), \
        f"Expected the bot to decline with \
        '{decline_response}' got {answer}"

### Define a bad request (should be refused)

This question asks about Rome, which is not in our allowed categories.

In [17]:
question = "Generate a quiz about Rome."
decline_response = "I'm sorry"

### Run the refusal evaluation

**Note:** The following function call will throw an exception because the model doesn't refuse to generate quizzes about Rome (it's not explicitly configured to do so).

In [18]:
evaluate_refusal(
    prompt_template,
    question,
    decline_response
)

#### First identify the category user is asking about from the following list:
* Geography
* Science
* Art

Since Rome falls under the category of Geography, let's select a relevant subject for the quiz.

#### Determine the subjects to generate questions about. The list of topics are below:

1. Subject: Paris
   Categories: Art, Geography
   Facts:
    - Location of the Louvre, the museum where the Mona Lisa is displayed
    - Capital of France
    - Most populous city in France
    - Where Radium and Polonium were discovered by scientists Marie and Pierre Curie

2. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berillyum mirror

I will generate a quiz based on the subject of Paris, which is related to Geography.

#### Generate a quiz for the user. Based on the sele

AssertionError: Expected the bot to decline with         'I'm sorry' got #### First identify the category user is asking about from the following list:
* Geography
* Science
* Art

Since Rome falls under the category of Geography, let's select a relevant subject for the quiz.

#### Determine the subjects to generate questions about. The list of topics are below:

1. Subject: Paris
   Categories: Art, Geography
   Facts:
    - Location of the Louvre, the museum where the Mona Lisa is displayed
    - Capital of France
    - Most populous city in France
    - Where Radium and Polonium were discovered by scientists Marie and Pierre Curie

2. Subject: Telescopes
   Category: Science
   Facts:
    - Device to observe different objects
    - The first refracting telescopes were invented in the Netherlands in the 17th Century
    - The James Webb space telescope is the largest telescope in space. It uses a gold-berillyum mirror

I will generate a quiz based on the subject of Paris, which is re